# Encryption

## Import libraries for AES-256 encryption/decryption

In [ ]:
import base64
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

## Encrypt data using AES-256

In [ ]:
text = "I \u2764\uFE0F Unicorns"
data = bytes(text, "utf-8")
print(data.decode("utf-8"))

KEY_SIZE_BITS = 256

encryption_key = get_random_bytes(KEY_SIZE_BITS // 8)
cipher = AES.new(encryption_key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(data)

with open("../data/secret.bin", "wb") as encrypted_file:
    encrypted_file.write(cipher.nonce)
    encrypted_file.write(tag)
    encrypted_file.write(ciphertext)

key = base64.b64encode(encryption_key)
print(key)

## Decrypt AES-256 data

In [ ]:
with open("../data/secret.bin", "rb") as encrypted_file:
    nonce = encrypted_file.read(16)
    tag = encrypted_file.read(16)
    ciphertext = encrypted_file.read()

decryption_key = base64.b64decode(key)
cipher = AES.new(decryption_key, AES.MODE_EAX, nonce)
plaintext = cipher.decrypt_and_verify(ciphertext, tag)

print(plaintext.decode("utf-8"))

## Generate public and private keys

In [ ]:
from Crypto.PublicKey import RSA

key = RSA.generate(2048)

private_key = key.export_key()
with open("../data/private.pem", "wb") as private_key_file:
    private_key_file.write(private_key)

public_key = key.publickey().export_key()
with open("../data/receiver.pem", "wb") as public_key_file:
    public_key_file.write(public_key)

## Import libraries for RSA encryption/decryption

In [ ]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes

## Encrypting data using RSA

In [ ]:
text = "I \u2764\uFE0F Rainbows"
data = text.encode("utf-8")

with open("../data/receiver.pem") as public_key_file:
    recipient_key = RSA.import_key(public_key_file.read())

session_key = get_random_bytes(16)

cipher_rsa = PKCS1_OAEP.new(recipient_key)
encrypted_session_key = cipher_rsa.encrypt(session_key)

cipher_aes = AES.new(session_key, AES.MODE_EAX)
ciphertext, tag = cipher_aes.encrypt_and_digest(data)

with open("../data/rsa_secret.bin", "wb") as encrypted_file:
    encrypted_file.write(encrypted_session_key + cipher_aes.nonce + tag + ciphertext)

## Decrypting data using RSA

In [ ]:
with open("../data/private.pem") as private_key_file:
    private_key = RSA.import_key(private_key_file.read())

with open("../data/rsa_secret.bin", "rb") as encrypted_file:
    encrypted_session_key = encrypted_file.read(private_key.size_in_bytes())
    nonce = encrypted_file.read(16)
    tag = encrypted_file.read(16)
    ciphertext = encrypted_file.read()

cipher_rsa = PKCS1_OAEP.new(private_key)
session_key = cipher_rsa.decrypt(encrypted_session_key)

cipher_aes = AES.new(session_key, AES.MODE_EAX, nonce)
data = cipher_aes.decrypt_and_verify(ciphertext, tag)

print(data.decode("utf-8"))

## Hashing

In [ ]:
from Crypto.Hash import SHA3_512

SALT = "Zpss8CTP"

password = "Rover & Mittens at #27"
data = bytes(SALT + password, "utf-8")

hash = SHA3_512.new()
hash.update(data)

print(hash.hexdigest())